__Основное задание:__
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая.



In [1]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
warnings.simplefilter("ignore")
%matplotlib inline

In [20]:
data = pd.read_csv("train.csv")

In [3]:
data.shape

(50001, 394)

__Задание 1:__ сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на тестовой выборке, оценить расхождение по сравнению с качеством на обучающей выборке и тестовой выборке.


In [4]:
x_train, x_test = train_test_split(
    data.drop(["TransactionID", "isFraud"], axis=1), train_size=0.666, shuffle=True, random_state=1,
)
y_train, y_test = train_test_split(
    data["isFraud"], train_size=0.666, shuffle=True, random_state=1,
)

In [5]:
numerical_features = x_train.select_dtypes(exclude=["object"])
numerical_features = numerical_features.columns.tolist()

x_train = x_train[numerical_features]
x_test = x_test[numerical_features]

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 33300 rows, 378 cols
x_test.shape = 16701 rows, 378 cols


In [6]:
model = xgb.XGBRegressor(random_state=1)
model.fit(x_train, y_train,  early_stopping_rounds=-1, eval_metric="auc", eval_set=[(x_train, y_train), (x_test, y_test)])

[0]	validation_0-auc:0.73568	validation_1-auc:0.70872
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in -1 rounds.
[1]	validation_0-auc:0.81086	validation_1-auc:0.78073
[2]	validation_0-auc:0.82044	validation_1-auc:0.80011
[3]	validation_0-auc:0.84156	validation_1-auc:0.81841
[4]	validation_0-auc:0.84694	validation_1-auc:0.82648
[5]	validation_0-auc:0.86482	validation_1-auc:0.84955
[6]	validation_0-auc:0.87516	validation_1-auc:0.86231
[7]	validation_0-auc:0.88434	validation_1-auc:0.86760
[8]	validation_0-auc:0.88724	validation_1-auc:0.86714
Stopping. Best iteration:
[7]	validation_0-auc:0.88434	validation_1-auc:0.86760



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
train_score = roc_auc_score(y_train, model.predict(x_train))
test_score = roc_auc_score(y_test, model.predict(x_test))

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.884, Test-score: 0.868


__Задание 2:__ сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1.

In [8]:
x_train, x_valid  = train_test_split(
    data.drop(["TransactionID", "isFraud"], axis=1), train_size=0.6666, shuffle=True, random_state=1,
)
y_train, y_valid = train_test_split(
    data["isFraud"], train_size=0.6666, shuffle=True, random_state=1,
)

x_train = x_train[numerical_features]
x_valid = x_valid[numerical_features]

x_valid, x_test = train_test_split(
    x_valid, train_size=0.6666, shuffle=True, random_state=27
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.6666, shuffle=True, random_state=27
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 33330 rows, 378 cols
x_valid.shape = 11112 rows, 378 cols
x_test.shape = 5559 rows, 378 cols


In [9]:
model = xgb.XGBRegressor(random_state=1,n_estimators=500)
model.fit(x_train, y_train, 
          early_stopping_rounds=-1, eval_metric="auc", eval_set=[(x_train, y_train), (x_valid, y_valid)])

[0]	validation_0-auc:0.75612	validation_1-auc:0.74284
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in -1 rounds.
[1]	validation_0-auc:0.81938	validation_1-auc:0.79439
[2]	validation_0-auc:0.82711	validation_1-auc:0.80912
[3]	validation_0-auc:0.83601	validation_1-auc:0.82251
[4]	validation_0-auc:0.86351	validation_1-auc:0.85001
[5]	validation_0-auc:0.86910	validation_1-auc:0.85534
[6]	validation_0-auc:0.87148	validation_1-auc:0.85743
[7]	validation_0-auc:0.88332	validation_1-auc:0.87088
[8]	validation_0-auc:0.89011	validation_1-auc:0.87399
[9]	validation_0-auc:0.89549	validation_1-auc:0.87783
[10]	validation_0-auc:0.89994	validation_1-auc:0.88103
[11]	validation_0-auc:0.90292	validation_1-auc:0.88312
[12]	validation_0-auc:0.90737	validation_1-auc:0.88358
[13]	validation_0-auc:0.90813	validation_1-auc:0.88483
[14]	validation_0-auc:0.90945	validation_1-auc:0.88489
[15]	validation_0-auc:0.9147

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(y_test, model.predict(x_test))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.916, Valid-score: 0.892, Test-score: 0.877


__Задание 3:__ построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на тестовом выборке относительно полученного доверительного интервала. Сделать выводы.



In [11]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [12]:
np.random.seed(27)
scores = create_bootstrap_metrics(y_test, model.predict(x_test), roc_auc_score)
calculate_confidence_interval(scores)

(0.8428974513645711, 0.9108542079421109)

__Задание 4:__ выполнить Adversarial Validation на основе данных из обучения / теста, подобрать объема из обучающей выборки, которые сильно похожи на объекты из тестовой выборки, и использовать их в качестве валидационного набора. Сделать выводы о полученных результатах.



In [16]:
x_adv = pd.concat([
    x_train, x_valid], axis=0)
y_adv = np.hstack((np.zeros(x_train.shape[0]), np.ones(x_valid.shape[0])))
assert x_adv.shape[0] == y_adv.shape[0]

In [17]:
model = xgb.XGBClassifier(n_estimators=25)
model.fit(x_adv[numerical_features], y_adv)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=25, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
y_pred = model.predict_proba(x_train[numerical_features])
#roc_auc_score()

In [19]:
pd.cut(
    y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]      102
(0.1, 0.2]     3186
(0.2, 0.3]    29018
(0.3, 0.4]      951
(0.4, 0.5]       63
(0.5, 0.6]       10
(0.6, 0.7]        0
(0.7, 0.8]        0
(0.8, 0.9]        0
(0.9, 1.0]        0
dtype: int64

__Задание 5:__ сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных;


In [13]:
cv = cross_val_score(
    estimator=model,
    X=data[numerical_features],
    y=data["isFraud"],
    scoring="roc_auc",
    cv=5
)

print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.8515 +/- 0.023


In [14]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [15]:
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    data[numerical_features], data["isFraud"], model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 40000, valid-observations = 10001
train-score = 0.919, valid-score = 0.82
Fold: 2, train-observations = 40001, valid-observations = 10000
train-score = 0.9168, valid-score = 0.8582
Fold: 3, train-observations = 40001, valid-observations = 10000
train-score = 0.9068, valid-score = 0.879
Fold: 4, train-observations = 40001, valid-observations = 10000
train-score = 0.909, valid-score = 0.8706
Fold: 5, train-observations = 40001, valid-observations = 10000
train-score = 0.8992, valid-score = 0.8761
CV-results train: 0.9102 +/- 0.007
CV-results valid: 0.8608 +/- 0.022
OOF-score = 0.8593



__Задание 6 * (опциональное):__ сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на тестовой выборке с полученным доверительным интервалом. Сделать выводы.



__Задание 7 ** (совсем опциональное):__ в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).

__Задание на повторение:__
Задание не обязательно к выполнению, но очень рекомендуется для понимания набора данных и получения гипотез для проведения валидации.   
__Задание 1:__ Построить график распределения времени совершения транзакции для обучающей / тестовой выборки, сделать выводы о том, как разбиты данные и какие виды валидации могут подойти для данной задачи.   
__Задание 2:__ построить scatter-plot зависимости суммы транзакции от времени совершения транзакции. Построить графики для обучающей выборки и для тестовой выборки, для обучающей выборки - построить как для целевой переменной = 0, так и для переменной = 1. Сделать выводы.   
__Задание 3:__ построить распределение признака TransactionAmt в логарифмическом масштабе, сделать выводы о близости распредления к нормальному распределению. Построить распределение признака в логарифмическому масштабе для обучающей выборк и для тестовой выборки, сделать выводы.  
__Задание 4:__ построить распределение признака целевой переменной в зависимости от значений категориальных признаков ProductCD, card4, card6. Сделать выводы.  
